Домашнее задание 1

import random
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
import random
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col
from pyspark.sql.functions import count, mean, max, min, expr, window, lag, when
import sys  # Добавляем импорт sys

In [3]:
random.seed(123)
random.randint(1, 200, ) # Номер варианта

14

In [4]:
# Шаг 1: Создание Spark Session и считывание файла
spark = SparkSession.builder.appName("DataProcessing").getOrCreate()


In [5]:
df = spark.read.csv("./144/1/1.csv", header=True, inferSchema=True)

In [6]:
# Шаг 2: Вывод первых 13 строк
df.show(13)

+----------+-----------------+-----+--------+-----------------+--------------------+---------------+---------------+------+--------------------+
|product_id|             name|price|quantity|         category|         description|production_date|expiration_date|rating|              status|
+----------+-----------------+-----+--------+-----------------+--------------------+---------------+---------------+------+--------------------+
|         1|              Сыр| 6779|     928|Молочные продукты|Помогает поддержи...|     1993-05-15|     2026-08-01|  3.51|    Awaiting Payment|
|         2|             Вода| 9374|     178|          Напитки|Широко использует...|     1990-04-08|     2013-02-12|  0.24|           Delivered|
|         3|         Патиссон| 4114|     877|            Овощи|Помогает поддержи...|     1991-01-16|     2020-04-02|   2.0|                 New|
|         4|         Нектарин| 7440|     157|           Фрукты|Отличается высоки...|     2020-06-01|     2022-06-01|  3.81|       

In [7]:
# Количество столбцов
len(df.columns)

10

In [8]:
statistic = df.describe()
statistic.show()

+-------+-----------------+---------------+------------------+------------------+--------+--------------------+------------------+--------------------+
|summary|       product_id|           name|             price|          quantity|category|         description|            rating|              status|
+-------+-----------------+---------------+------------------+------------------+--------+--------------------+------------------+--------------------+
|  count|             1230|           1230|              1230|              1230|    1230|                1230|              1230|                1230|
|   mean|            615.5|           NULL|  5109.29512195122|501.31463414634146|    NULL|                NULL|2.4986991869918724|                NULL|
| stddev|355.2147237939328|           NULL|2873.4760426765765| 294.4435985707421|    NULL|                NULL|1.4384995003861554|                NULL|
|    min|                1|Adrenaline Rush|               101|                 1| Бакале

In [9]:
df = df.drop("description")

In [10]:
df.columns

['product_id',
 'name',
 'price',
 'quantity',
 'category',
 'production_date',
 'expiration_date',
 'rating',
 'status']

In [11]:
# Вычисление общей суммы цены
total_price = df.select("price").agg({"price": "sum"}).collect()[0][0]

# Добавляем новый столбец с нормализованными ценами
df = df.withColumn("normalized_price", 
                    when(col("price") == 0, 0).otherwise(col("price") / total_price))

In [12]:
# Проверяем результат
df.show()

+----------+--------------------+-----+--------+-----------------+---------------+---------------+------+--------------------+--------------------+
|product_id|                name|price|quantity|         category|production_date|expiration_date|rating|              status|    normalized_price|
+----------+--------------------+-----+--------+-----------------+---------------+---------------+------+--------------------+--------------------+
|         1|                 Сыр| 6779|     928|Молочные продукты|     1993-05-15|     2026-08-01|  3.51|    Awaiting Payment|0.001078697155336...|
|         2|                Вода| 9374|     178|          Напитки|     1990-04-08|     2013-02-12|  0.24|           Delivered|0.001491622235450...|
|         3|            Патиссон| 4114|     877|            Овощи|     1991-01-16|     2020-04-02|   2.0|                 New|6.546334410757502E-4|
|         4|            Нектарин| 7440|     157|           Фрукты|     2020-06-01|     2022-06-01|  3.81|       

In [16]:
# Если хотите добавить заголовок (имена столбцов)
df.write.option("header", True).csv("r_1.csv")

Py4JJavaError: An error occurred while calling o73.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:6